In [ ]:
# Install pyspark
! pip install --ignore-installed pyspark

# Install Spark NLP
! pip install --ignore-installed spark-nlp

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.training import CoNLL

print("Spark NLP version", sparknlp.version())

Spark NLP version 3.2.2


In [ ]:
spark = sparknlp.start()
spark

To configure MFA we just need to define the requires values in spark properties as show below. Look an example to get temporal credentials [here](https://github.com/JohnSnowLabs/spark-nlp/blob/master/scripts/aws_tmp_credentials.sh) 

In [ ]:
spark.conf.set("spark.jsl.settings.aws.credentials.access_key_id", "MY_ACCESS_KEY_ID")
spark.conf.set("spark.jsl.settings.aws.credentials.secret_access_key", "MY_SECRET_ACCESS_KEY_ID")
spark.conf.set("spark.jsl.settings.aws.credentials.session_token", "MY_SESSION_TOKEN")
spark.conf.set("spark.jsl.settings.aws.region", "MY_REGION")

In [ ]:
1from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, 'sample_data/test_ner_dataset.txt')
training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|John Smith works ...|[{document, 0, 35...|[{document, 0, 35...|[{token, 0, 3, Jo...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [ ]:
embeddings = WordEmbeddingsModel.pretrained("glove_100d")
ready_data = embeddings.transform(training_data).cache()

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
graphFolder = "s3://my.bucket.com/my/s3/path"

ner_tagger = NerDLApproach() \
                .setInputCols("sentence", "token", "embeddings") \
                .setLabelColumn("label") \
                .setOutputCol("ner") \
                .setMinEpochs(1) \
                .setMaxEpochs(30) \
                .setRandomSeed(0) \
                .setVerbose(0) \
                .setDropout(0.8) \
                .setBatchSize(18) \
                .setGraphFolder(graphFolder)

In [ ]:
ner_tagger.fit(ready_data)

NerDLModel_18c6a5b33e9a